In [51]:
def readfile(filepath):
    with open(filepath) as f: 
        str_text = f.read()
    return str_text

In [52]:
def separate_punc(doc_text):
        return[token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n' ]

In [53]:
t = readfile('moby_dick_four_chapters.txt')

In [54]:
import spacy
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from pickle import dump,load

In [55]:
nlp = spacy.load("en_core_web_sm",disable=['parser', 'tagger','ner'])

nlp.max_length = 1198623

In [56]:
d = readfile('melville-moby_dick.txt')

In [57]:
token = separate_punc(t)
token

/home/victor/.local/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on',
 'shore',
 'i',
 'thought',
 'i',
 'would',
 'sail',
 'about',
 'a',
 'little',
 'and',
 'see',
 'the',
 'watery',
 'part',
 'of',
 'the',
 'world',
 'it',
 'is',
 'a',
 'way',
 'i',
 'have',
 'of',
 'driving',
 'off',
 'the',
 'spleen',
 'and',
 'regulating',
 'the',
 'circulation',
 'whenever',
 'i',
 'find',
 'myself',
 'growing',
 'grim',
 'about',
 'the',
 'mouth',
 'whenever',
 'it',
 'is',
 'a',
 'damp',
 'drizzly',
 'november',
 'in',
 'my',
 'soul',
 'whenever',
 'i',
 'find',
 'myself',
 'involuntarily',
 'pausing',
 'before',
 'coffin',
 'warehouses',
 'and',
 'bringing',
 'up',
 'the',
 'rear',
 'of',
 'every',
 'funeral',
 'i',
 'meet',
 'and',
 'especially',
 'whenever',
 'my',
 'hypos',
 'get',
 'such',
 'an',
 'upper',
 'hand',
 '

In [58]:
len(token)

11338

In [59]:
# the the NN predct the 26th word
train_length = 25 + 1
text_sequnce = []

for i in range(train_length, len(token)):
    seq = token[i -train_length : i]
    text_sequnce.append(seq)
    

In [60]:
text_sequnce[1]

['me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on',
 'shore']

In [61]:
' '.join(text_sequnce[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [62]:
tokenier = Tokenizer()
tokenier.fit_on_texts(text_sequnce)
tokenier.texts_to_sequences(text_sequnce)

[[956,
  14,
  263,
  51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24],
 [14,
  263,
  51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957],
 [263,
  51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5],
 [51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5,
  60],
 [261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5,
  60,
  5],
 [408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5,
  60,
  5,
  5

In [63]:
sequences = tokenier.texts_to_sequences(text_sequnce)

In [64]:
vocabulary_size = len(tokenier.word_counts)

In [65]:
vocabulary_size

2718

In [66]:
sequences = np.array(sequences)

In [67]:
X = sequences[:,:-1]
Y = sequences[:,-1]

In [68]:
y = to_categorical(Y, num_classes= vocabulary_size+1)
seq_len= X.shape[1]
X.shape


(11312, 25)

In [69]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [70]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 25, 25)            67975     
                                                                 
 lstm_4 (LSTM)               (None, 25, 150)           105600    
                                                                 
 lstm_5 (LSTM)               (None, 150)               180600    
                                                                 
 dense_4 (Dense)             (None, 150)               22650     
                                                                 
 dense_5 (Dense)             (None, 2719)              410569    
                                                                 
Total params: 787,394
Trainable params: 787,394
Non-trainable params: 0
_________________________________________________________________


2023-09-23 15:12:17.690256: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-23 15:12:17.691423: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-23 15:12:17.692317: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [71]:
X = sequences[:,:-1]
y = sequences[:,-1]
model.fit(X, y, batch_size=128, epochs=3,verbose=1)

Epoch 1/3


2023-09-23 15:12:18.028995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-23 15:12:18.030133: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-23 15:12:18.030984: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

ValueError: in user code:

    File "/home/victor/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/victor/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/victor/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/victor/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/victor/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/home/victor/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_